In [76]:
class Vektor:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        x, y = self.koordinate()
        return ('{}(x = {}, y = {})'.format(self.__class__.__name__, x, y))
    
    def __add__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 + x2, y1 + y2))
    
    def __sub__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 - x2, y1 - y2))
    
    def __rmul__(self, konstanta):
        x1 = konstanta * self.x
        y1 = konstanta * self.y
        return(Vektor(x1, y1))
    
    def koordinate(self):
        return self.x, self.y
    
class Premica:
    def __init__(self, u1, u2): # u tocka; rabili bomo koordinate
        u1x, u1y = u1.koordinate()
        u2x, u2y = u2.koordinate()
        if u2x - u1x != 0:
            self.koef = (u2y -u1y) / (u2x - u1x)
            self.n = u1y - self.koef * u1x
        else:
            self.koef = "NaN"
            self.x = u1x
        
    def parametri(self):
        if self.koef != "NaN":
            return self.koef, self.n
        else:
            return self.x
    
    def __repr__(self):
        if self.koef != "NaN":
            k, n = self.parametri()
            return ('{}(y = {} * x + {})'.format(self.__class__.__name__, k, n))
        else:
            x = self.parametri()
            return ('{}(x = {})'.format(self.__class__.__name__, x))
    
class U:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def sosedi(self):
        s = set({V(self.a, self.b), V(self.a, self.b - 1), V(self.a + 1, self.b - 1)})
        return s
    
    def na(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y == premica.koef * x + premica.n:
                return True
            else:
                return False
        # navpicnica:
        else:
            if x == premica.x:
                return True
            else:
                return False

class V:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2 + Vektor(1, 0)
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def sosedi(self):
        s = set({U(self.a, self.b), U(self.a - 1, self.b + 1), U(self.a, self.b + 1)})
        return s
        
    def na(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y == premica.koef * x + premica.n:
                return True
            else:
                return False
        # navpicnica:
        else:
            if x == premica.x:
                return True
            else:
                return False
            
        

A1 = Vektor(0, - sqrt(3))
A2 = Vektor(3 / 2, - sqrt(3) / 2)

def grid(k, l, m, n):
    # oglisca = ([0, 0], [k, l], [m, n], [k + m, l + n])
    # print U(0,0).koordinate(), U(k, l).koordinate(), U(m, n).koordinate(), U(k + m, l + n).koordinate()
    # lahko bi dali vsa vozlisca kar v mnozico??
    mina = min(0, k, m, k + m)
    maxa = max(0, k, m, k + m)
    minb = min(0, l, n, l + n)
    maxb = max(0, l, n, l + n)
    
    #matrika = []
    #for i in range(mina, maxa + 1):
    #    vrstica = []
    #    for j in range(minb, maxb + 1):
    #        vrstica += [U(i, j)]
    #        # print U(i,j), U(i, j).koordinate()
    #    matrika += [vrstica]
    #    
    #return matrika
    
    mnozica = set()
    for i in range(mina, maxa + 1):
        for j in range(minb, maxb + 1):
            mnozica.add(U(i, j))
    
    return mnozica
    

from matplotlib.path import Path
# mogoce to celo dela?? # trenutno vrne vse tocke v paralelogramu. odstraniiti je potrebno se tiste na obeh premicah
# premisliti je treba se tiste pogoje, ce je use uredu... to, da lezi na premici pa to
# (sem zdej naredu, lih prej k sm su u lj, pa nism preverju)
def paralelogram(k, l, m, n):
    G = grid(k, l, m, n)
    oglisca = [U(0,0).koordinate(), U(k, l).koordinate(), 
               U(k + m, l + n).koordinate(), U(m, n).koordinate()]
    p = Path(oglisca[::-1])
    premica1 = Premica(U(0, 0), U(k, l))
    premica2 = Premica(U(0, 0), U(m, n))
    Uji = set()
    Vji = set()
    for u in G:
        izbranaU = u
        a = izbranaU.a
        b = izbranaU.b
            
        izbranaV = V(a, b)
        tockaU = [izbranaU.koordinate()]
        tockaV = [izbranaV.koordinate()]
        # pogoji
        vsebuje_u = p.contains_points(tockaU, radius = 1e-2)[0] # ker vrne array[True]
        vsebuje_v = p.contains_points(tockaV, radius = 1e-2)[0]
        if (vsebuje_u) and (not izbranaU.na(premica1)) and (not izbranaU.na(premica2)): 
            Uji.add(izbranaU)
        elif (vsebuje_v) and (not izbranaV.na(premica1)) and (not izbranaV.na(premica2)):
            Vji.add(izbranaV)
            
    return Uji, Vji

In [63]:
paralelogram(-2,0,0,2) # ne vsebuje tocke V(-1,1)
#s.contains_points([V(-1,1).koordinate()], radius = 1e-2)

({U(-1a, 2b), U(-2a, 2b), U(-1a, 1b), U(-2a, 1b)},
 {V(0a, 0b), V(-1a, 0b), V(0a, 1b)})

In [77]:
V(-1,1).koordinate()
U(0,0).sosedi()

{V(1a, -1b), V(0a, 0b), V(0a, -1b)}